# Overview

Let's create three different collections, and query it at once using LlamaIndex.


More information about this: https://docs.llamaindex.ai/en/stable/module_guides/indexing/index_guide/

In [1]:
# let's catch some logs
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Connect to Weaviate

In [2]:
import weaviate
from weaviate import classes as wvc
  
# Connect to a local, embedded instance
client = weaviate.connect_to_embedded()

INFO:weaviate-client:Started /Users/dudanogueira/.cache/weaviate-embedded: process ID 83300
Started /Users/dudanogueira/.cache/weaviate-embedded: process ID 83300


{"action":"startup","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-11-13T15:16:51-03:00"}
{"action":"startup","auto_schema_enabled":true,"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-11-13T15:16:51-03:00"}
{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-11-13T15:16:51-03:00"}
{"build_git_commit":"ab0312d5d","build_go_versio

INFO:httpx:HTTP Request: GET http://localhost:8079/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
HTTP Request: GET http://localhost:8079/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8079/v1/meta "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8079/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8079/v1/.well-known/ready "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8079/v1/.well-known/ready "HTTP/1.1 200 OK"


{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","docker_image_tag":"localhost","level":"info","msg":"configured versions","server_version":"1.26.6","time":"2024-11-13T15:16:53-03:00"}
{"action":"grpc_startup","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"grpc server listening at [::]:50050","time":"2024-11-13T15:16:53-03:00"}
{"address":"192.168.28.127:51826","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"current Leader","time":"2024-11-13T15:16:53-03:00"}
{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"starting migration from old schema","time":"2024-11-13T15:16:53-03:00"}
{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","b

INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"


{"action":"telemetry_push","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"telemetry started","payload":"\u0026{MachineID:aaa598be-f7d1-48c4-9981-8eabbeee03a0 Type:INIT Version:1.26.6 NumObjects:0 OS:darwin Arch:arm64 UsedModules:[]}","time":"2024-11-13T15:16:53-03:00"}


In [ ]:
# lets check the connection getting the server version
print(f"Client: {weaviate.__version__}, Server: {client.get_meta().get('version')}")

INFO:httpx:HTTP Request: GET http://localhost:8079/v1/meta "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8079/v1/meta "HTTP/1.1 200 OK"
Client: 4.9.3, Server: 1.26.6


## Create Collections 

### Blog Post Collections

In [4]:
from weaviate import classes as wvc
# clean slate
client.collections.delete("BlogPost")

collection = client.collections.create(
    name="BlogPost",
    description="Blog post from the Weaviate website.",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(
        model="text-embedding-3-small"
    ),
    generative_config=wvc.config.Configure.Generative.openai(
        model="gpt-3.5-turbo"
    ),
    properties=[
        wvc.config.Property(name="text", description="Content from the blog post", data_type=wvc.config.DataType.TEXT)
    ]
)

print("Collection was created.")

INFO:httpx:HTTP Request: DELETE http://localhost:8079/v1/schema/BlogPost "HTTP/1.1 200 OK"
HTTP Request: DELETE http://localhost:8079/v1/schema/BlogPost "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8079/v1/schema "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:8079/v1/schema "HTTP/1.1 200 OK"
Collection was created.


{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"warning","msg":"prop len tracker file /Users/dudanogueira/.local/share/weaviate/blogpost/Yenb2lylXSIW/proplengths does not exist, creating new tracker","time":"2024-11-13T15:17:01-03:00"}
{"action":"hnsw_prefill_cache_async","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"not waiting for vector cache prefill, running in background","time":"2024-11-13T15:17:01-03:00","wait_for_cache_prefill":false}
{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"Created shard blogpost_Yenb2lylXSIW in 3.126083ms","time":"2024-11-13T15:17:01-03:00"}
{"action":"hnsw_vector_cache_prefill","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":

### Podcast Collections

In [5]:
from weaviate import classes as wvc
# clean slate
client.collections.delete("PodCast")

collection = client.collections.create(
    name="PodCast",
    description="Weaviate podcast",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(
        model="text-embedding-3-small"
    ),
    generative_config=wvc.config.Configure.Generative.openai(
        model="gpt-3.5-turbo"
    ),
    properties=[
        wvc.config.Property(name="text", description="Content from podcasts", data_type=wvc.config.DataType.TEXT)
    ]
)

print("Collection was created.")

INFO:httpx:HTTP Request: DELETE http://localhost:8079/v1/schema/PodCast "HTTP/1.1 200 OK"
HTTP Request: DELETE http://localhost:8079/v1/schema/PodCast "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8079/v1/schema "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:8079/v1/schema "HTTP/1.1 200 OK"
Collection was created.


{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"warning","msg":"prop len tracker file /Users/dudanogueira/.local/share/weaviate/podcast/PgqgLoHorvKd/proplengths does not exist, creating new tracker","time":"2024-11-13T15:17:03-03:00"}
{"action":"hnsw_prefill_cache_async","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"not waiting for vector cache prefill, running in background","time":"2024-11-13T15:17:03-03:00","wait_for_cache_prefill":false}
{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"Created shard podcast_PgqgLoHorvKd in 3.057791ms","time":"2024-11-13T15:17:03-03:00"}
{"action":"hnsw_vector_cache_prefill","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1

In [6]:
from weaviate import classes as wvc
# clean slate
client.collections.delete("MeetingNotes")

collection = client.collections.create(
    name="MeetingNotes",
    description="Weaviate Meeting Notes",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(
        model="text-embedding-3-small"
    ),
    generative_config=wvc.config.Configure.Generative.openai(
        model="gpt-3.5-turbo"
    ),
    properties=[
        wvc.config.Property(name="text", description="Content from meeting notes", data_type=wvc.config.DataType.TEXT)
    ]
)

print("Collection was created.")

INFO:httpx:HTTP Request: DELETE http://localhost:8079/v1/schema/MeetingNotes "HTTP/1.1 200 OK"
HTTP Request: DELETE http://localhost:8079/v1/schema/MeetingNotes "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8079/v1/schema "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:8079/v1/schema "HTTP/1.1 200 OK"
Collection was created.


{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"warning","msg":"prop len tracker file /Users/dudanogueira/.local/share/weaviate/meetingnotes/AvU2hkhsYsdf/proplengths does not exist, creating new tracker","time":"2024-11-13T15:17:08-03:00"}
{"action":"hnsw_prefill_cache_async","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"not waiting for vector cache prefill, running in background","time":"2024-11-13T15:17:08-03:00","wait_for_cache_prefill":false}
{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"Created shard meetingnotes_AvU2hkhsYsdf in 3.211ms","time":"2024-11-13T15:17:08-03:00"}
{"action":"hnsw_vector_cache_prefill","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_vers

## Load Data

### Load blogs

In [7]:
from llama_index.core import SimpleDirectoryReader

blogs = SimpleDirectoryReader('./data').load_data()

In [8]:
# Print an example
print(blogs[0].text)


title: Multi-Tenancy Vector Search with millions of tenants


Large-scale setups were always a great reason to choose Weaviate. Last year we wrote about the first time [a Weaviate setup ran with a billion objects & vectors](/blog/sphere-dataset-in-weaviate). What was a mere experiment back then is a regular production case today. But earlier this year, we saw a shift in usage patterns: As we onboarded more and more large-scale and enterprise users, the definition of scale shifted from the number of vectors to the number of individual tenants that can run on a single setup.

Previously, Weaviate offered multiple ways to tackle multi-tenancy, but none were intended for a massive scale. Weaviate `v1.20` - coming in July 2023 - changes this once and for all: Native multi-tenancy support that scales to millions of tenants with 10s of thousands of active tenants per node. Yet scale is not the only point that makes the new multi-tenancy feature great; we put a lot of emphasis on compliance a

### Load podcasts (sticking to the release podcasts only)
let's first install the youtube-transcript-reader dependecy

In [ ]:
%pip install llama-index-readers-youtube-transcript

In [10]:
from llama_index.readers.youtube_transcript import YoutubeTranscriptReader

loader = YoutubeTranscriptReader()
podcasts = loader.load_data(ytlinks=['https://www.youtube.com/watch?v=xk28RMhRy1U&t=2302s', 'https://www.youtube.com/watch?v=Du6IphCcCec', 
'https://www.youtube.com/watch?v=Q7f2JeuMN7E&t=578s', 'https://www.youtube.com/watch?v=nSCUk5pHXlo&t=22s'])

In [11]:
# let's get one example
podcasts[0].text

"thank you so much for watching the wevia\npodcast I'm super excited to welcome we\nV8 CTO and co-founder Eddie and dilocker\nfor the weeviate 1.20 release podcast\nthis is another packed release Abate\nwith all sorts of cool things\nmulti-tenancy PQ rescoring Auto cut\nre-rankers a new hybrid uh rank Fusion\nalgorithm and some Cloud monitoring\nmetrics every time we do these release\npodcasts it's always so much fun I\nalways learn so much so Eddie and\nfirstly thank you so much for joining\nthe podcast\nthanks so much for having me same for me\nI love talking about these things I love\nthe the great questions that you always\nprepare and I'm very very excited as\nwell to talk about this one\nyeah amazing yeah I love just like the\nbreadth of it going through all the\ndifferent topics in weaviate and so\nstarting off with I think just a super\nexciting topic diving into the database\nthing multi-tenancy can you tell us uh\nmaybe just to begin in the highest level\nabstraction like the

### Load Meeting Notes Index

In [12]:
from llama_index.core import SimpleDirectoryReader

meeting_notes = SimpleDirectoryReader('./meeting-notes').load_data()

In [13]:
# print example
print(meeting_notes[0].text)

Meeting Notes
Date: July 13, 2023
Attendees: Connor (C), Weaviate (W)

Agenda: Exploring Multi-Tenancy with Weaviate's Vector Database

Introduction and Goals of Meeting (B):

Connor explained that the primary goal of the meeting was to discuss how Weaviate's vector database could be integrated into Connor's platform, a photo sharing platform, with a specific focus on multi-tenancy.
Explanation of Vector Databases (W):

Weaviate provided an overview of vector databases, focusing on how they handle high-dimensional data points and efficiently perform similarity searches.
Benefits of Multi-Tenancy (W):

Weaviate explained the concept of multi-tenancy, highlighting its potential for resource sharing, cost reduction, and ease of maintenance. They mentioned that their vector database is capable of supporting multi-tenant architecture.
Connors's Interest in Multi-Tenancy (B):

Connor confirmed the importance of multi-tenancy for their platform. Given that they serve many different users and 

## Build the Indices

### Import libs

In [14]:
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core import VectorStoreIndex, StorageContext, Settings
from llama_index.embeddings.openai import OpenAIEmbedding
import openai
import os

# global
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Lets set the OPENAI key
# os.environ["OPENAI_API_KEY"] = "sk-key"
openai.api_key = os.environ["OPENAI_API_KEY"]

### Blogs Index

In [15]:
# Let's name our index properly as BlogPost, as we will need it later.
vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name="BlogPost"
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
blogs_index = VectorStoreIndex.from_documents(
    blogs, storage_context=storage_context
)

INFO:httpx:HTTP Request: GET http://localhost:8079/v1/schema/BlogPost "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8079/v1/schema/BlogPost "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8079/v1/schema "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8079/v1/schema "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: GET http://localhost:8079/v1/nodes "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8079/v1/nodes "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8079/v1/nodes "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8079/v1/nodes "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8079/v1/nodes "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8079/v1/nodes "HTTP/1.1 200 OK"


### Podcast Index

In [16]:
# Let's name our index properly as BlogPost, as we will need it later.
vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name="PodCast"
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
podcasts_index = VectorStoreIndex.from_documents(
    blogs, storage_context=storage_context
)

INFO:httpx:HTTP Request: GET http://localhost:8079/v1/schema/PodCast "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8079/v1/schema/PodCast "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


### Meeting Notes Index

In [17]:
# Let's name our index properly as BlogPost, as we will need it later.
vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name="MeetingNotes"
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
notes_index = VectorStoreIndex.from_documents(
    meeting_notes, storage_context=storage_context
)

INFO:httpx:HTTP Request: GET http://localhost:8079/v1/schema/MeetingNotes "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8079/v1/schema/MeetingNotes "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


## Summary of each Index

In [18]:
from llama_index.core import DocumentSummaryIndex

In [19]:
blogs_index_summary = """
This index contains all of the blog posts that are on Weaviate.io.
"""

In [20]:
podcasts_index_summary = """
This index contains the Weaviate podcasts about new releases.
"""

In [21]:
meetingnotes_index_summary = """
This index contains notes from a client named Connor.
"""

In [22]:
index_summaries = [blogs_index_summary, podcasts_index_summary, meetingnotes_index_summary]
blogs_index.set_index_id("blogs_index")
podcasts_index.set_index_id("podcasts_index")
notes_index.set_index_id("notes_index")

## Query Time

In [23]:
from llama_index.core.indices.composability import ComposableGraph
from llama_index.core.indices import SummaryIndex

graph = ComposableGraph.from_indices(
    SummaryIndex,
    [blogs_index, podcasts_index, notes_index],
    index_summaries=index_summaries
)

In [24]:
custom_query_engines = {
    graph.root_id: graph.root_index.as_query_engine(
        retriever_mode="default" )
}

query_engine = graph.as_query_engine(
    custom_query_engines=custom_query_engines,
)

In [25]:
response = query_engine.query(
    "What is multi-tenancy? Why is it an important feature for Connor's application?"
)

print(str(response))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8079/v1/schema/BlogPost "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8079/v1/schema/BlogPost "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8079/v1/schema/BlogPost "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8079/v1/schema/BlogPost "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8079/v1/schema/PodCast "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8079/v1/schema/PodCast "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8079/v1/schema/PodCast "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8079/v1/schema/PodCast "HTTP/1.1 200 OK"
INFO:httpx:HTT

In [26]:
print(response.get_formatted_sources())

> Source (Doc id: 52d45997-c5bb-4386-9269-99880ae8cb34): Multi-tenancy is the ability to serve multiple distinct users or user groups from a single applic...

> Source (Doc id: 285431a7-6bd2-4242-b9b6-45fd29f99804): Multi-tenancy is the capability to serve multiple distinct users or user groups from a single app...

> Source (Doc id: 1dff9708-e905-4df3-9487-df514ba45c0b): Multi-tenancy is the concept of hosting multiple separate instances or groups of users within a s...

> Source (Doc id: 62b0e1c6-a364-4d29-a3fa-69b8cc8921cf): ### Using filters
Another approach was to use a single class and use Weaviate’s built-in filterin...

> Source (Doc id: 5b56877b-0a7c-481f-a549-4841961087ea): title: Multi-Tenancy Vector Search with millions of tenants


Large-scale setups were always a gr...

> Source (Doc id: 4cc589ef-6ed8-4273-8810-0e9feb779ca9): ### Using filters
Another approach was to use a single class and use Weaviate’s built-in filterin...

> Source (Doc id: a837c5c6-62ae-4195-975d-11d9

In [ ]:
client.close()